In [62]:
import xml.etree.ElementTree as ET
import pandas as pd
import sqlalchemy as sa

with open('../manifest_jan_2019.xml', 'r') as f:
    mani = ET.parse(f)

In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
root = mani.getroot()
def parse_element(elem):
    return_dict = {}
    for e in elem:
        return_dict[e.tag] = e.text
    return return_dict
def parse_root(root):
        return [parse_element(child) for child in iter(root) if child.tag != 'timestamp']

filedf = pd.DataFrame(parse_root(root))
filedf[['num_items', 'size']] = filedf[['num_items', 'size']].astype(int)
filedf['filename'] = filedf['filename'].astype(str)

In [65]:
import databases.create_db_define_models as cre

In [91]:
eng = sa.create_engine('sqlite:///arxiv1.db') #, echo=True)
eng.connect()

In [104]:
for index, line in filedf.iterrows():
    #line = filedf.iloc[50*k]
    f = cre.ManifestTarFile(**line)
    sess.add(f)
    sess.commit()

In [98]:
cre.ManifestTarFile.metadata.create_all(eng)

In [96]:
SMaker = sa.orm.sessionmaker(bind=eng)
sess = SMaker()

In [85]:
sess.add(f)

In [86]:
sess.commit()

2019-02-02 14:23:16,121 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-02 14:23:16,126 INFO sqlalchemy.engine.base.Engine INSERT INTO manifest (filename, content_md5sum, md5sum, first_item, last_item, num_items, seq_num, size, timestamp, yymm) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-02-02 14:23:16,129 INFO sqlalchemy.engine.base.Engine ('src/arXiv_src_0006_001.tar', 'b5bea3821e15af75b831250e6ea0a031', 'ea268ff2bc89161c6998146fbb2d7515', 'astro-ph0006001', 'quant-ph0006136', 2438, '1', 242543048, '2010-12-23 00:34:56', '0006')
2019-02-02 14:23:16,134 INFO sqlalchemy.engine.base.Engine COMMIT
